In [ ]:
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

In [ ]:
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews",
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA7G6A3/imdb_reviews-train.tfrecord…

Generating test examples...:   0%|          | 0/25000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA7G6A3/imdb_reviews-test.tfrecord*…

Generating unsupervised examples...:   0%|          | 0/50000 [00:00<?, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteA7G6A3/imdb_reviews-unsupervised.t…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [ ]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [ ]:
train_example_batch,train_labels_batch=next(iter(train_data.batch(10)))

In [ ]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [ ]:
train_labels_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [ ]:
hub_layer = hub.KerasLayer("https://kaggle.com/models/google/gnews-swivel/frameworks/TensorFlow2/variations/tf2-preview-20dim/versions/1", output_shape=[20],
                           input_shape=[], dtype=tf.string)


In [ ]:
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 20)                400020    
                                                                 
 dense (Dense)               (None, 16)                336       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 400373 (1.53 MB)
Trainable params: 353 (1.38 KB)
Non-trainable params: 400020 (1.53 MB)
_________________________________________________________________


In [ ]:
model.compile(optimizer='adam',loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['accuracy'])

In [ ]:
history=model.fit(train_data.shuffle(1000).batch(100),epochs=8,validation_data=validation_data.batch(100), verbose=1)

Epoch 1/8


/usr/local/lib/python3.10/dist-packages/keras/src/backend.py:5818: UserWarning: "`binary_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Sigmoid activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


150/150 [==============================] - 4s 22ms/step - loss: 0.6631 - accuracy: 0.6303 - val_loss: 0.6080 - val_accuracy: 0.6683
Epoch 2/8
150/150 [==============================] - 3s 19ms/step - loss: 0.5980 - accuracy: 0.6825 - val_loss: 0.5882 - val_accuracy: 0.6881
Epoch 3/8
150/150 [==============================] - 5s 31ms/step - loss: 0.5837 - accuracy: 0.6930 - val_loss: 0.5812 - val_accuracy: 0.6980
Epoch 4/8
150/150 [==============================] - 3s 20ms/step - loss: 0.5748 - accuracy: 0.7029 - val_loss: 0.5700 - val_accuracy: 0.7053
Epoch 5/8
150/150 [==============================] - 3s 20ms/step - loss: 0.5683 - accuracy: 0.7055 - val_loss: 0.5665 - val_accuracy: 0.7118
Epoch 6/8
150/150 [==============================] - 4s 25ms/step - loss: 0.5635 - accuracy: 0.7111 - val_loss: 0.5626 - val_accuracy: 0.7148
Epoch 7/8
150/150 [==============================] - 3s 19ms/step - loss: 0.5608 - accuracy: 0.7147 - val_loss: 0.5589 - val_accuracy: 0.7138
Epoch 8/8
150/15

In [ ]:
results=model.evaluate(test_data.batch(100),verbose=2)

250/250 - 4s - loss: 0.5631 - accuracy: 0.7110 - 4s/epoch - 15ms/step


In [ ]:
results

[0.5630798935890198, 0.7110400199890137]